# Step 0-1: Get Data & S2 input data 

***

## Step 0: Get Data

### 0.1 Get S2 from GEE

In GEE run code to get Sentinel-2 imagery without clouds for September 2, 2021.

Link to GEE Code: https://code.earthengine.google.com/06e0b9733609d890d5bdd43df538f71d

### 0.2 Get S2 & other data from Drive

After GEE code is run, S2 imagery will be exported into Google Drive. 

Link to project data folder: https://drive.google.com/drive/folders/1O_ZBg3JLkzNLwHJUhyCU2UZ5-rb6ZJJW?usp=share_link

In folder:

- Sentinel2_09012021.tif (S2 data WITHOUT cloud mask)
- Philadelphia_dem_3ft_2022.tif (study area DEM 3ft resolution)
- Philadelphia.geojson (study area)
- Phila_Buff_5k.geojson (buffered study area)

### 0.3 Make DEM Inputs

In QGIS use the DEM file to create slope and hillshade layers, then combine them all into one layer. 

In QGIS use the raster tool "slope" and "hillshade" on the DEM layer. Then use the "merge" tool (under Miscelaneous) to combine the slope and hillshade layers (click "add each layer as seperate band") then repeat to add the resulting merged layer to the DEM layer. Note which bands are which (1-hillshade, 2-slope, 3-dem), reprojected to 32618, updated metadata & exported as a geotiff.

***

## Step 1: Make S2 input data

### 1.1 Get data into python

If having trouble importing, check Anaconda, if not in there then open Anaconda Prompt (go to where the app is), then type --> conda install -c conda-forge PACKAGENAME

In [47]:
# Import packages
import os
import rasterio as rio
from rasterio.plot import show
import rioxarray as rxr
import geopandas as gpd
import numpy as np
import pandas as pd
import folium
import matplotlib.pyplot as plt

In [48]:
# Set wd
wd_path = "C:\\Users\\rcompos\\OneDrive - North Carolina State University\\Documents\\Research\\Scripts"
os.chdir(wd_path)
os.getcwd() # get wd
#os.listdir() # get available data

'C:\\Users\\rcompos\\OneDrive - North Carolina State University\\Documents\\Research\\Scripts'

### 1.2 Prepare Sentinel-2

In [13]:
# Paths
s2_path = "./Sentinel_data/Sentinel2_09022021_4326_clip.tif" # Sentinel2_09012021.tif
#phi_path = "Phila_Buff_5k.geojson" # ".\\Phila_shapes\\Files\\Philadelphia.shp" #"Phila_Buff_5k.geojson" 

In [ ]:
# Save needed bands for calculations
s2 = rio.open(s2_path) # read in data
one = s2.read(1)
blue = s2.read(2)
green = s2.read(3)
red = s2.read(4)
five = s2.read(5)
six = s2.read(6)
sev = s2.read(7)
nir = s2.read(8)
nin = s2.read(9)
swir1 = s2.read(11) # SWIR1 is band 11, SWIR2 is band 12
swir2 = s2.read(12)
thtee = s2.read(13)
s2.close()
#mir = ???

In [ ]:
#s2.crs

## 1.3 Calculations with Sentinel-2

#### Normalized Difference Vegetation Index (NDVI)
Formula: NDVI = (NIR-Red)/(NIR+Red)

##### NDVI Test 1
Uses planet code - https://developers.planet.com/docs/planetschool/calculate-an-ndvi-in-python/
And github - https://github.com/parulnith/Satellite-Imagery-Analysis-with-Python/blob/master/Calculating%20NDVI%20for%20our%20Area%20of%20Interest.ipynb

In [ ]:
# Calculate NDVI
# Planet recommends multiplying bands by coefficients made from TOA

# Allow division by zero
np.seterr(divide='ignore', invalid='ignore')

# Run Formula 
ndvi = (nir.astype(float) - red.astype(float)) / (nir + red)

# Explore
#ndvi # print
#np.nanmax(ndvi) # get max
show(source = ndvi, ax = None) # display

##### NDVI Test 2
Use BGU code - https://geobgu.xyz/py/rasterio.html#reading-raster-data

##### NDVI Test 3 
Uses Earthpy

#### Enhanced Vegetation Index (EVI)
Formula: 2.5 * (NIR - RED) / ((NIR + 6* RED - 7.5* BLUE) + 1

About: Considered the advanced ndvi, should be between 1 (green) and 0 (not healthy veg) (BUT landsat says it can be between -10k and 10k). Improved bc incorporate more bands that decrease background + atmospheric noise, + saturation. 


Sources: https://www.usgs.gov/landsat-missions/landsat-enhanced-vegetation-index;
https://custom-scripts.sentinel-hub.com/custom-scripts/sentinel-2/evi/#; https://hub4everybody.com/enhanced-vegetation-index-evi/?hs-panel=&hs-x=1883916.5156348734&hs-y=6303928.185111765&hs-z=11&hs-lang=en&hs-visible-layers=OpenStreetMap%3BEVI%202020_04_22%3BEVI%202020_05_22%3Brostenice_2020&map-swipe=disabled&app=default


EVI = G * ((NIR - R) / (NIR + C1 * R – C2 * B + L))
In Landsat 4-7, EVI = 2.5 * ((Band 4 – Band 3) / (Band 4 + 6 * Band 3 – 7.5 * Band 1 + 1)).

In [ ]:
# Calculate EVI

# Run Formula 
evi = 2.5 * ((nir - red) / (nir + 6*red - 7.5*blue + 1))

# Explore
#np.nanmax(evi)
#np.min(evi)
#show(evi)

#### Normalized Difference Water Index (NDWI)
Formula: (Green-NIR)/(Green+NIR)

In [ ]:
# Calculate NDWI

# Run Formula 
ndwi = (green - nir) / (green + nir)

# Explore
#np.nanmax(ndwi)
#show(ndwi)

#### Modified Normalized Difference Water Index (MNDWI)
Formula: MNDWI = (Green-MIR)/(Green+MIR)

In [ ]:
# Calculate MNDWI --> Is it possible without MIR?

# Run Formula 
#mndwi = (green - mir) / (green + mir)

# Explore
#np.nanmax(mndwi)
#show(mndwi)

#### Normalized Difference Moisture Index (NDMI)
Formula: NDMI = (NIR-SWIR1)/(NIR+SWIR1)

In [ ]:
# Calculate NDMI

# Run Formula 
ndmi = (nir - swir1) / (nir + swir1)

# Explore
#np.nanmax(ndmi)
#show(ndmi)

#### Automated Water Extraction Index (AWEI)
Formulas: 
AWEInsh = 4(Green-SWIR1)-(.25NIR+2.75SWIR2) 
AWEIsh = blue + 2.5

Description: "AWEInsh is an index formulated to effectively eliminate nonwater pixels, including dark built surfaces in areas with urban background and AWEIsh is primarily formulated for further improvement of accuracy by removing shadow pixels that AWEInsh may not effectively eliminate. But in areas with highly reflective surfaces such as ice, snow and reflective roofs in urban areas, (aweish) may misclassify such surfaces as water."

Source: https://www.sciencedirect.com/science/article/pii/S0034425713002873?ref=pdf_download&fr=RR-2&rr=79e31ae5efb2b0c9

In [ ]:
# Calculate AWEI

# Run Formula 
awei = (4*(green - swir1))-(.25*nir + 2.75*swir2)

# Explore
#np.nanmax(awei)
#show(awei)

## Prepare DEM Data

# CLOSE EVERYTHING SO U DON'T GET HACKED

In [59]:
src.close()
s2.close()
dem.close()

NameError: name 's2' is not defined